In [481]:
import os
from glob import glob
import pandas as pd
from IPython.display import display
import numpy as np

# raw data

In [482]:
path_list=glob(os.path.join(os.getcwd(),'dataset','*csv*')) 
path_list

['/media/disk3/feynman52/ntu-ml/hw1/dataset/train.csv',
 '/media/disk3/feynman52/ntu-ml/hw1/dataset/test.csv']

https://www.ptt.cc/bbs/Python/M.1479880647.A.47D.html

In [483]:
df_train = pd.read_csv(path_list[0],encoding = 'big5')
# df_train

# row order by datetime

In [484]:
feature_list = df_train['測項'].unique()
feature_list

array(['AMB_TEMP', 'CH4', 'CO', 'NMHC', 'NO', 'NO2', 'NOx', 'O3', 'PM10',
       'PM2.5', 'RAINFALL', 'RH', 'SO2', 'THC', 'WD_HR', 'WIND_DIREC',
       'WIND_SPEED', 'WS_HR'], dtype=object)

In [485]:
feature_num = len(feature_list)

i=1
start = i*feature_num
end = (i+1)*feature_num

In [486]:
df_train[start:end]

In [487]:
date = df_train[start:end].iloc[0,0]
date

'2014/1/2'

In [488]:
datetime_list = pd.date_range(start=date, periods=24, freq='60min')
datetime_list

DatetimeIndex(['2014-01-02 00:00:00', '2014-01-02 01:00:00',
               '2014-01-02 02:00:00', '2014-01-02 03:00:00',
               '2014-01-02 04:00:00', '2014-01-02 05:00:00',
               '2014-01-02 06:00:00', '2014-01-02 07:00:00',
               '2014-01-02 08:00:00', '2014-01-02 09:00:00',
               '2014-01-02 10:00:00', '2014-01-02 11:00:00',
               '2014-01-02 12:00:00', '2014-01-02 13:00:00',
               '2014-01-02 14:00:00', '2014-01-02 15:00:00',
               '2014-01-02 16:00:00', '2014-01-02 17:00:00',
               '2014-01-02 18:00:00', '2014-01-02 19:00:00',
               '2014-01-02 20:00:00', '2014-01-02 21:00:00',
               '2014-01-02 22:00:00', '2014-01-02 23:00:00'],
              dtype='datetime64[ns]', freq='60T')

In [489]:
df_dummy = df_train[start:end].T[3:]
df_dummy.columns = feature_list
df_dummy['time'] = datetime_list
df_dummy

In [490]:
df = pd.DataFrame([])
i = 0
while True:
    start = i*feature_num
    end = (i+1)*feature_num
    if end > len(df_train): break
#     if i == 3: break
        
    date = df_train[start:end].iloc[0,0]
    datetime_list = pd.date_range(start=date, periods=24, freq='60min')
    
    df_dummy = df_train[start:end].T[3:]
    df_dummy.columns = feature_list
    df_dummy['time'] = datetime_list
    
#     display(df_dummy)
    df = df.append(df_dummy)
    
    i+=1 
    
df

In [491]:
col_list = list(df.columns)
col_list = col_list[-1:] + col_list[:-1]
col_list
df = df[col_list]
# df

In [492]:
def return_val(x):
    if x == 'NR': return 0
    else: return float(x)
    
for col in df.columns:
    if col != 'time':
        df[col] = df[col].apply(lambda x: return_val(x))
# df

In [493]:
df = df.reset_index(drop=True)
# df

# slice one month then slice group by group

In [494]:
day_per_month = 20
hr_per_day = 24
hr_per_month = day_per_month*hr_per_day
hr_per_group = 10

x_train = []
y_train = []

for i_month in range(0,len(df),hr_per_month):
    
#     if i_month>0: break
    
    start_month = i_month
    end_month = i_month+hr_per_month
#     print(start_month,end_month)
    df_one_month = df[start_month:end_month]
    
#     display(df_one_month)

    start_group = 0
    while True:
        
#         if start_group > 3: break
        
        end_group = start_group+hr_per_group
        if end_group > len(df_one_month): break
            
        df_one_group = df_one_month.iloc[start_group:end_group,:]
#         display(df_one_group)
        start_group+=1
        
        x = df_one_group.iloc[:-1,1:].values
        y = df_one_group['PM2.5'].values[-1]
        
        x = x.T
        
        x_train.append(x)
        y_train.append(y)
    

x_train = np.array(x_train)    
y_train = np.array(y_train)    
    
    
x_train.shape,y_train.shape

((5652, 18, 9), (5652,))

In [495]:
x_train = x_train.reshape(len(x_train),-1)
print(x_train.shape)
bias=np.ones(shape=(len(x_train),1))
print(x_train.shape)
x_train = np.concatenate([x_train,bias],axis=1)
print(x_train.shape)

(5652, 162)
(5652, 162)
(5652, 163)


# traing

In [496]:
learning_rate = 10
epochs = 100
gradient_square_sum = np.zeros( shape=x_train.shape[1] )
w=np.zeros(shape=x_train.shape[1])

In [497]:
for epoch in range(epochs):
    y_hat = x_train @ w
    delta_y = y_hat - y_train
    gradient = x_train.T @ delta_y

    gradient_square_sum = gradient_square_sum + gradient ** 2
    ada = np.sqrt(gradient_square_sum)
    new_gradient = gradient/ada
    w = w - learning_rate*new_gradient

    mse = (np.sum(delta_y**2))/len(delta_y)
    rmse = np.sqrt(mse)
    print('--------')
    print(epoch,rmse,mse)


--------
0 27.071214829194115 732.8506723283793
--------
1 50528.58459434335 2553137861.1077123
--------
2 27.066936884635428 732.6190723168378
--------
3 14.762751444990597 217.93883022657195
--------
4 14.541508115635352 211.45545827708884
--------
5 14.32995312616457 205.34755659807374
--------
6 14.127694167067741 199.59174247819985
--------
7 13.934353816162963 194.1662162740153
--------
8 13.749565962276545 189.05056415099372
--------
9 13.572975656282201 184.22566816602927
--------
10 13.404238967831501 179.67362230673254
--------
11 13.243022845405347 175.37765408392792
--------
12 13.089004977529333 171.32205130178764
--------
13 12.941873653225596 167.49209365605483
--------
14 12.801327620015698 163.87398883497676
--------
15 12.66707593803817 160.45481282002558
--------
16 12.538837829095216 157.22245410434923
--------
17 12.416342519691046 154.1655615662878
--------
18 12.299329077365021 151.2734957533167
--------
19 12.187546239852127 148.53628334853371
--------
20 12.080

# testing

In [498]:
# save model
np.save('model.npy',w)
# read model
w = np.load('model.npy')

In [499]:
feature_list

array(['AMB_TEMP', 'CH4', 'CO', 'NMHC', 'NO', 'NO2', 'NOx', 'O3', 'PM10',
       'PM2.5', 'RAINFALL', 'RH', 'SO2', 'THC', 'WD_HR', 'WIND_DIREC',
       'WIND_SPEED', 'WS_HR'], dtype=object)

In [500]:
feature_num

18

In [501]:
df_test = pd.read_csv(path_list[1],encoding = 'big5',header=None)
df_test.columns = ['id','feature']+['hr%s'%(i) for i in range(1,10)]
# df_test

In [502]:
df_test.iloc[3,3]

'0.09'

In [503]:
for col in df_test.columns:
    if col not in ['id','feature']:
        df_test[col] = df_test[col].apply(lambda x: return_val(x))

In [504]:
df_test.iloc[3,3]

0.09

In [505]:
# 18 (feature_num) row per group

x_test = []
for i_group in range(0,len(df_test),feature_num):
    
#     if i_group>100: break
    
    start_group = i_group
    end_group = start_group + feature_num
    
    df_group = df_test[start_group:end_group]
    
#     display(df_group)
    x=df_group.iloc[:,2:].values
    x_test.append(x)

x_test = np.array(x_test)
    

In [506]:
x_test.shape

(240, 18, 9)

In [507]:
x_test.shape

(240, 18, 9)

In [508]:
x_test = x_test.reshape(len(x_test),-1)
x_test.shape

(240, 162)

In [509]:
bias_test=np.ones(shape=(len(x_test),1))
bias_test.shape

(240, 1)

In [510]:
x_test = np.concatenate([x_test,bias_test],axis=1)
x_test.shape

(240, 163)

In [511]:
w.shape

(163,)

In [512]:
y_test = x_test @ w
y_test.shape

(240,)

In [513]:
a=pd.DataFrame(x_test[:,81:81+9])
a['y'] = y_test
a

,0,1,2,3,4,5,6,7,8,y
0,27.0,13.0,24.0,29.0,41.0,30.0,29.0,27.0,28.0,22.237237
1,46.0,47.0,57.0,78.0,84.0,76.0,59.0,61.0,61.0,62.911587
2,10.0,10.0,25.0,34.0,40.0,39.0,36.0,25.0,22.0,33.656937
3,71.0,58.0,51.0,41.0,41.0,46.0,43.0,34.0,29.0,34.352978
4,13.0,23.0,18.0,10.0,5.0,5.0,13.0,9.0,12.0,10.512915
5,25.0,32.0,40.0,40.0,31.0,27.0,13.0,24.0,29.0,25.001748
6,37.0,42.0,65.0,70.0,71.0,58.0,51.0,41.0,41.0,44.396819
7,17.0,22.0,17.0,9.0,17.0,23.0,29.0,17.0,18.0,18.668087
8,78.0,84.0,76.0,59.0,61.0,61.0,60.0,49.0,49.0,61.802023
9,13.0,13.0,18.0,20.0,14.0,9.0,16.0,22.0,35.0,23.049980
